In [1]:
from wine_classification_hyperparameter_search_v2 import (
    get_dataframe,
    create_search_grid,
    train_model,
    compare_model_results,
    analyze_model
)

%load_ext autoreload
%autoreload 2

/Users/tchase/anaconda3/envs/union-poc-env/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


15:12:19.319480 WARNING  type_engine.py:1358 - Unsupported Type typing.Any      
                         found, Flyte will default to use PickleFile as the     
                         transport. Pickle can only be used to send objects     
                         between the exact same version of Python, and we       
                         strongly recommend to use python type that flyte       
                         support.

In [2]:
grid = {
    'max_depth': [10, 50, 100],
    'max_features': [None, 'sqrt'],
    'n_estimators': [100, 2000]
}

In [3]:
dataframe = get_dataframe()
dataframe.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [4]:
hyperparameters = create_search_grid(grid = grid)
hyperparameters

[{'max_depth': 10, 'max_features': None, 'n_estimators': 100},
 {'max_depth': 10, 'max_features': None, 'n_estimators': 2000},
 {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100},
 {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 2000},
 {'max_depth': 50, 'max_features': None, 'n_estimators': 100},
 {'max_depth': 50, 'max_features': None, 'n_estimators': 2000},
 {'max_depth': 50, 'max_features': 'sqrt', 'n_estimators': 100},
 {'max_depth': 50, 'max_features': 'sqrt', 'n_estimators': 2000},
 {'max_depth': 100, 'max_features': None, 'n_estimators': 100},
 {'max_depth': 100, 'max_features': None, 'n_estimators': 2000},
 {'max_depth': 100, 'max_features': 'sqrt', 'n_estimators': 100},
 {'max_depth': 100, 'max_features': 'sqrt', 'n_estimators': 2000}]

In [5]:
from functools import partial
from flytekit import map_task

models = map_task(partial(train_model, dataframe=dataframe))(hyperparameters=hyperparameters)
print(len(models))

12


In [6]:
best_model = compare_model_results(models = models, hyperparameters=hyperparameters, dataframe = dataframe, force_plot=True)

In [7]:
analyze_model(best_model, dataframe = dataframe, force_plot=True)